In [ ]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import subprocess
from datetime import date
import math

# Converting .mov videos to .mp4
def conv_mp4(input_file):
    out_name = os.path.splitext(input_file)[0].lstrip('.') 
    print(out_name)
    output_file = out_name + '.mp4'

    # Run FFmpeg command to convert MOV to MP4
    command = f"ffmpeg -i {input_file} -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 128k {output_file}"
    subprocess.call(command, shell=True)
    new = os.path.join(os.path.dirname(os.path.dirname(output_file)), 'Conv_vids', os.path.basename(output_file))
    shutil.move(output_file, new)


def downsize(path, to_path):
    dsv = deeplabcut.DownSampleVideo(path, width=320, height=240)
    shutil.move(dsv, os.path.join(to_path, os.path.basename(dsv)))

#cropping video and Gamma correction
def gamma_correct(convid):
    
    # Load the video file
    video = cv2.VideoCapture(convid)

    # Get the frame rate and total number of frames
    fps = int(video.get(cv2.CAP_PROP_FPS))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    #*********************************************
    # Set appropriate values for target average illumination of the frame
    target_br = 0.6

    #**********************************************
    
    target_g = math.log(target_br)
    
    # Set the video writer
    output_name = os.path.splitext(os.path.basename(convid))[0].lstrip('.') + '_AGC.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_name, fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    start_frame = 0
    
    # Set the current frame number to the start frame
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    frame_number = start_frame
    
    # Loop through the frames and write them to the output video
    for i in range(start_frame, total_frames):
        ret, frame = video.read()
        if not ret:
            break
        
        # Normalize the pixel values to the range [0, 1]
        frame_normalized = frame.astype(np.float32) / 255.0
        gray = cv2.cvtColor(frame_normalized, cv2.COLOR_BGR2GRAY)
        
        # Finding the mean luminance of the frame
        brightness = np.mean(frame_normalized)

        #finding and correcting gamma - a number is assigned to be the 'innate' gamma value of the frame, and that value is used to find the correction needed. 
        if brightness == 0:
            print('err')
            continue
        assigned_g = math.log(brightness)
        gamma = assigned_g/target_g

        # Apply gamma correction
        frame_corrected = np.power(frame_normalized, 1/gamma)

        # Scale the pixel values back to the range [0, 255]
        frame_scaled = (frame_corrected * 255.0).astype(np.uint8)
        
        print(round(frame_number/total_frames*100, 2), end = '\r')
        frame_number += 1
        # cv2.imshow('output', frame)
        # cv2.imshow('output', frame_scaled)
        out.write(frame_scaled)
    # new = os.path.join(os.path.dirname(os.path.dirname(convid)), 'GC_Conv_vids', os.path.basename(convid))
    # shutil.move(dsv, new)
    print("Done - ", convid)
    print('')
    # Release the video objects
    video.release()
    out.release()
    cv2.destroyAllWindows()

#cropping video and Gamma correction
def adap_gamma_correct(convid, tbr, clp, gs):
    
    # Load the video file
    video = cv2.VideoCapture(convid)

    # Get the frame rate and total number of frames
    fps = int(video.get(cv2.CAP_PROP_FPS))
    # fps = 30
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(total_frames)

    #*********************************************

    target_br = tbr
    target_g = math.log(target_br)

    gs_w, gs_h = gs

    #**********************************************

    # Set the video writer
    output_name = os.path.splitext(os.path.basename(convid))[0].lstrip('.') + '_' + str(int(target_br*100)) + '_' + str(clp) + '_' + str(gs_w) + '_' + str(gs_h) + '_AGC_clahe.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_name, fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    start_frame = 0
    
    # Set the current frame number to the start frame
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    frame_number = 0
    
    # Loop through the frames and write them to the output video
    for i in range(start_frame, total_frames):
        ret, frame = video.read()
        if not ret:
            break
        
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
                
        clahe = cv2.createCLAHE(clipLimit=clp, tileGridSize=(gs_w, gs_h))
        lab[...,0] = clahe.apply(lab[...,0])
        
        frame_scaled_edt = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        
        
        frame_normalized = frame_scaled_edt.astype(np.float32) / 255.0
        gray = cv2.cvtColor(frame_normalized, cv2.COLOR_BGR2GRAY)
        
        brightness = np.mean(frame_normalized)
        
        #finding and correcting gamma - a number is assigned to be the 'innate' gamma value of the frame, and that value is used to find the correction needed. 
        if brightness == 0:
            # cv2.imshow('output', frame)
            print(brightness, ' - err')
            continue
        assigned_g = math.log(brightness)
        gamma = assigned_g/target_g
        
        # Apply gamma correction
        frame_corrected = np.power(frame_normalized, 1/gamma)

        # Scale the pixel values back to the range [0, 255]
        frame_scaled = (frame_corrected * 255.0).astype(np.uint8)        

        progress = frame_number/total_frames*100
        print(round(progress, 2), noted, end = '\r')
        frame_number += 1

        out.write(frame_scaled)
    video.release()
    out.release()
    cv2.destroyAllWindows()
    print("Done - ", convid)
    print('')


#Processing videos using DeepLabCut ModelZoo's pretrained model - Primate face
def process(fold, project_name, your_name):
    file = fold[0]
    print(file)
    bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
                 'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
    videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
    video_down = folder
    name_fold = 'Analyse_' + os.path.basename(os.path.dirname(file))
    
    model_options = deeplabcut.create_project.modelzoo.Modeloptions
    model_selection = 'primate_face'

    config_path, train_config_path = deeplabcut.create_pretrained_project(
        project_name,
        your_name,
        video_down,
        videotype=videotype,
        model=model_selection,
        analyzevideo=True,
        createlabeledvideo=False,
        copy_videos=False,
    )

    edits = {
        'dotsize': 1.5,  # size of the dots!--------------------------------------------------------------------------------------------------------------------------------
        'pcutoff': 0.4,  # the higher, the more conservative the plotting!
    }
    deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)
    project_path = os.path.dirname(config_path)
    full_video_path = []
    for i in fold:
        full_video_path.append(os.path.join(project_path,'videos', os.path.basename(i)))

    # filter predictions (should already be done above ;):
    deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

    # re-create the video with your edits!
    # deeplabcut.CropVideo(config_path, full_video_path, 'crop', 
    deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)

#Cropping the video based on confidence of facial features
def crop_it(feed):
    
    fl_pth, vid_pth, x = feed
    # deeplabcut.analyze_videos(config_path, 'CropGr.mp4', save_as_csv=True, dynamic=(True,.6,30))
    print("Starting Cropping")

    # Load tracking results generated by DeepLabCut
    tracking_data = pd.read_hdf(fl_pth)

    # Define the names of the facial features that you want to extract frames for
    feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

    # Define the threshold for the confidence score of the facial features
    confidence_threshold = 0.9
    
    # Load the input video
    cap = cv2.VideoCapture(vid_pth)
    
    tot_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define the output video writer
    bn = os.path.basename(vid_pth).split('.')[0]
    out_file = 'Crpoutput_video' + bn + '.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_file, fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    print("Output file written - ", out_file)

    # Loop through the video frames and extract frames with facial features
    frame_number = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            # Get the tracking data for the current frame
            frame_data = tracking_data.loc[frame_number]

            # Check if the desired facial features are present in the frame
            feature_present = False
            check = 0
            for feature_name in feature_names:
                if feature_name in frame_data.loc[x] and frame_data.loc[x].loc[feature_name].loc['likelihood'] > confidence_threshold:
                    check += 1
            if check==4:
                feature_present = True

            # If the desired facial features are present, save the frame to the output video
            if feature_present: #check==4
                # cv2.imshow('output', frame)
                out.write(frame)
            # Display the output
            #cv2.imshow('output', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            print(round(frame_number/tot_frame*100, 2), end = '\r')
            # Increment the frame number
            frame_number += 1
        else:
            break
    print("Done")
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def main():
    #enter path of folder (including folder name) containing all your videos of interest------------------------------------------------------------------------------------------
    vid_dir = '/home/yramakrishna/DeepLabCut/conda-environments/Videos'
    #initializing project name and author's name ---------------------------------------------------------------------------------------------------------------------------------
    project_name = 'DLC'
    your_name = 'YR'
    
    os.chdir(os.path.dirname(vid_dir))
    print(os.getcwd())
    
    #Create a new folder called All_vids in the current directory
    if not os.path.exists('All_vids'):
            os.makedirs('All_vids')
    os.chdir('All_vids')
    
    to_dir = os.path.join(os.getcwd(), os.path.basename(vid_dir)) #moving the folderwith videos to the current directory
    shutil.move(vid_dir, to_dir)
    
    #Storing converted videos in a new folder
    if not os.path.exists('Conv_vids'):
            os.makedirs('Conv_vids')
            
    
    #Converting files from mov to MP4 (if any)
    for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Videos')):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Videos', file))
        if file.endswith(".mov"):
            conv_mp4(path)
        elif file.endswith(".mp4"):
            path2=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
            shutil.copy(path, path2)
    
    os.chdir(os.path.dirname(to_dir))
    
    if not os.path.exists('Downsize_Conv_vids'):
            os.makedirs('Downsize_Conv_vids')
    os.chdir('Downsize_Conv_vids')
    
    # vid_names = []
    for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Conv_vids')):
        if file.endswith(".mp4"):
            path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
            downsize(path, os.getcwd())
    
    os.chdir(os.path.dirname(to_dir))
    
    #Creae a folder for gamma corrected videos
    if not os.path.exists('AGC_Conv_vids'):
            os.makedirs('AGC_Conv_vids')
    os.chdir('AGC_Conv_vids')
    
    #Applying adaptive gamma correction with CLAHE on the mp4 videos
    for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Downsize_Conv_vids')):
        if file.endswith(".mp4"):
            path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Downsize_Conv_vids', file))
            gamma_correct(path)
            video = cv2.VideoCapture(path)
            width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            tbr = 0.72
            clp = 2
            gs_w = int(width / 64.0)
            gs_h = int(height / 48.0)
            gs = [gs_w, gs_h]
            adap_gamma_correct(path, tbr, clp, gs)
            
    os.chdir('..')
    
    #Compiling a list ofpaths for analysis
    folder = [] 
    for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'AGC_Conv_vids')):
        if file.endswith(".mp4"):
            path=os.path.join(os.path.join(os.path.dirname(to_dir), 'AGC_Conv_vids', file))
            folder.append(path)
    
    #Analyzing all videos with ModelZoo
    process(folder, project_name, your_name)
    
    #Collecting files with information on labels, as well as the labelled videos.
    os.chdir(os.path.join(os.getcwd()))
    h5files = []
    vid_to_crop = []
    today = str(date.today())
    # today = "2023-04-12" -------------------------------------------------Modify if you created the modelzoo files on a different day than today.
    proj_fold = project_name+'-'+your_name+'-'+today
    target = os.path.join(os.getcwd(),proj_fold,'videos')
    for file in os.listdir(target):
        if file.endswith("filtered.h5"):
            h5files.append(os.path.join(target,file))
    for file in os.listdir(target):   
        if file.endswith("labeled.mp4"):
            vid_to_crop.append(os.path.join(target,file))
    
    
    os.chdir(os.path.join(os.getcwd()))
    if not os.path.exists('Cropped_vids'):
            os.makedirs('Cropped_vids')
    os.chdir('Cropped_vids')
    
    crop_source = []
    
    if len(h5files)==len(vid_to_crop):
        for i in h5files:
            found = ''
            for j in vid_to_crop:
                x = os.path.splitext(i)[0].rstrip('_filtered.h5')
                y = os.path.splitext(j)[0].rstrip('filtered_labeled.mp4')
                if x==y:
                    found = j
            k = os.path.basename(i)
            l = 'DLC' + k.split('DLC')[1] + 'DLC' + k.split('DLC')[2].rstrip('_filtered.h5')
            crop_source.append([i,found, l])
    
    c_s = np.array(crop_source)
    
    all_frames = []
    
    for i in c_s:
        x = crop_it(i)
        all_frames.append([i[1],x])

if __name__ == "__main__":
    main()